# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

### Wcode Class

In [5]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  #---
  def show()
    self.values.map(&:to_s).join
  end
  def to_a()
    self.values.flatten
  end
  def size()
    self.to_a.size
  end
  def copy()
    wc = self.class.new
    wc.each_key{|key| wc[key] = self[key].dup}
    wc
  end
  #---
  def narrow()
    wc = self.copy
    sorted = wc.values.flatten.sort
    wc.each_key do |key|
      wc[key].map!{|v| sorted.index(v)}
    end
    wc
  end
  #---
  def to_word()
    chars = (0..self.size/2).to_a.map{|i| self.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  #---
  def get_four_tuple(odd_nums)
    arr = [odd_nums[0], odd_nums[0]+1, odd_nums[1], (odd_nums[1]+1)%self.size]
    FourTuple.new(self.to_a.select{|n| arr.include?(n)}, self.size)
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}] }.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}] }.to_h
    facs.map! do |f|
      wc = self.class.new
      wc.each_key{|key| wc[key] = f[key]}
    end
    return facs
  end
  #---
end; nil

#-----------------------------------
class FourTuple < Array
  def initialize(arr, mod)
    @modulo = mod
    4.times{|i| self[i] = arr[i]}
    @odd_nums = self.select{|n| n.odd?}.sort
  end
  
  def linking()
    odd1 = @odd_nums[0]
    if (self.index(odd1) - self.index(odd1+1) + 1).even?
      # [o,e,*,*] / [o,*,*,e] / [*,o,e,*] / [*,*,o,e] (and reverse 'o' and 'e')
      return 0
    else
      # [o1,e2,e1,o2] --> linking = +1 and [o2,o1,e2,e1] --> linking = +1
      # The sign of linking is change by (o1<->e1) and (e2<->o2)
      idxs = [self.index(odd1), self.index(odd1+1)] 
      sign0 = (idxs[0] < idxs[1]) ? 1 : -1
      rng = idxs.sort
      sign2 = self[rng[0]..rng[1]][1].odd? ? 1 : -1
      #---
      sign = sign0*sign2
    end
  end
end;nil

### Turaev cobraket

In [6]:
def cobracket(myw)
  wcode = Wcode.new(myw)
  odd_pairs=(1..wcode.size/2).to_a.map{|x| 2*x-1}.combination(2)
  #--------------------------------------
  coprods = odd_pairs.map do |odds|
    order = wcode.get_four_tuple(odds)
    factors = wcode.devide(odds)
    prewords = factors.map(&:narrow).map(&:to_word)
    words = prewords.map(&:cyclic_reduce)
    #---
    {order: order,
     linking: order.linking, 
     factors: factors, 
     prewords: prewords,
     words: words
     }
  end
  monomials = coprods.select{|prd| prd[:linking] != 0}
  i = 0
  result = []
  while monomials.size > 0
    pop = monomials.pop
    coeff = pop[:linking]
    indices = []
    coeff += monomials.map do |mono|
      if mono[:words] == pop[:words]
        indices << monomials.index(mono)
        mono[:linking]
      elsif mono[:words] == pop[:words].reverse
        indices << monomials.index(mono)
        mono[:linking]*(-1)
      else
        0
      end
      end.sum
    #---
    term = pop[:words].join(" \u{2227} ")
    if coeff != 0 
      result << ((coeff == 1) ? '' : "(#{coeff.to_s}) ") + term
    end
    #---
    indices.each{|i| monomials.delete_at(i)}
  end 
  return (result.empty?) ? [0] : result
end;nil

### Execution

In [10]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten
samples = [
  a1, 
  a1*Word.new(b[2]),
  a1*(Word.new(b[2]).inverse),
  #---
  a1^4,
  c,
  comms[1],
  #---
  a1*(comms[2].inverse),
  a1*comms[2],
  a1*(comms[2]^3),
  a1*(a2^3),
  a1*(Word.new(a[2], b[2].inverse)^3),
  a1*comms[2]*c*comms[2]*(c.inverse),
  #---
  a1*Group.commutator(a[2], a[3]),
  a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ]

samples.each do |s|
  ws = s.show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  p "#{ws}  |--\u{03B4}-->  #{cobracket(s).join(' + ')}"
end;nil

"a  |--δ-->  0"
"a.d  |--δ-->  a ∧ d"
"a.D  |--δ-->  0"
"a.a.a.a  |--δ-->  (-1) aa ∧ aa"
"abA  |--δ-->  (-1) 1 ∧ b"
"abAB  |--δ-->  0"
"a.dcDC  |--δ-->  a ∧ dcDC"
"a.cdCD  |--δ-->  0"
"a.cdCD.cdCD.cdCD  |--δ-->  (-1) acdCDcdCD ∧ cdCD + (-1) acdCD ∧ cdCDcdCD"
"a.c.c.c  |--δ-->  (-1) acc ∧ c + (-1) ac ∧ cc"
"a.cD.cD.cD  |--δ-->  (-1) acDcD ∧ cD + (-1) acD ∧ cDcD"
"a.cdCD.abA.cdCD.aBA  |--δ-->  (-1) acdCD ∧ cdCD + (-1) cdCD ∧ cdCDa"
"a.ceCE  |--δ-->  aceC ∧ E + (-1) ace ∧ CE + ac ∧ C"
"a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  acdCDefEFceCfeFEdcDC ∧ E + (-1) acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"


In [12]:
myw = a1*(comms[2].inverse)*a2*(Word.new(b[1]).inverse)
wcode = Wcode.new(myw)
p myw.show, wcode.show

odd_pairs=(1..wcode.size/2).to_a.map{|x| 2*x-1}.combination(2)
#odd_pairs = odd_pairs.map{|p| [p, p.reverse]}.flatten(1)
#odd_pairs = [[1,3],[3,1]]

#--------------------------------------
coprods = odd_pairs.map do |odds|
  order = wcode.get_four_tuple(odds)
  factors = wcode.devide(odds)
  prewords = factors.map(&:narrow).map(&:to_word)
  words = prewords.map(&:cyclic_reduce)
  #---
  {order: order,
   linking: order.linking, 
   factors: factors, 
   prewords: prewords,
   words: words
   }
end
#--------------------------------------

#--------------------------------------
coprods.select{|h| h[:linking] != 0 }.each do |h|
  printf(".....\n")
  p "#{h[:order]}"
#  p "#{h[:linking]}"
  p h[:factors].map(&:show).join(" , ")
#  p h[:prewords].join("(x)")
#  p ((h[:linking]==1) ? "+" : "-") + h[:words].join("(x)") + \
#   ((h[:linking]!=1) ? "+" : "-") + h[:words].reverse.join("(x)")
  p [h[:linking] , h[:words].join(" \u{2227} ")]
end
#--------------------------------------

nil

"(((a)(dcDC))(c))(B)"
"[0][12][1][13][9, 4, 10][3, 6][11, 5, 8][7, 2][][][][]"
.....
"[12, 1, 11, 2]"
"[0][12][1][13][][][][][][][][] , [][][][][9, 4, 10][3, 6][11, 5, 8][7, 2][][][][]"
[1, "aB ∧ c"]
.....
"[0, 1, 13, 2]"
"[0][][1][][][][][][][][][] , [][12][][13][9, 4, 10][3, 6][11, 5, 8][7, 2][][][][]"
[1, "a ∧ dcDB"]
.....
"[9, 4, 10, 3]"
"[0][12][1][13][10][3][11][2][][][][] , [][][][][9, 4][6][5, 8][7][][][][]"
[1, "adcB ∧ D"]
.....
"[12, 6, 11, 5]"
"[0][12][1][13][4][3][5][2][][][][] , [][][][][9, 10][6][11, 8][7][][][][]"
[-1, "adcB ∧ D"]
.....
"[0, 12, 13, 11]"
"[0][][1][][9, 4, 10][3, 6][11, 5, 8][7, 2][][][][] , [][12][][13][][][][][][][][]"
[-1, "adcD ∧ B"]


In [9]:
monomials = coprods.select{|prd| prd[:linking] != 0}

i = 0
result = []
while monomials.size > 0
  pop = monomials.pop
  coeff = pop[:linking]
  indices = []
  coeff += monomials.map do |mono|
    if mono[:words] == pop[:words]
      indices << monomials.index(mono)
      mono[:linking]
    elsif mono[:words] == pop[:words].reverse
      indices << monomials.index(mono)
      mono[:linking]*(-1)
    else
      0
    end
    end.sum
  #---
  term = pop[:words].join(" \u{2227} ")
  if coeff != 0 
    result << ((coeff == 1) ? '' : coeff.to_s + "*") + term
  end
  #---
  indices.each{|i| monomials.delete_at(i)}
end 
  
p result
  
nil

NoMethodError: private method `select' called for nil:NilClass

### TODO
$[a_{1},b_{1}]$ の処理において、
* [x] [5, 0, 6, 7] の sign が正しくないようだ。
* [x] a(x)A [ $a_{1}\otimes a_{1}^{-1}$ ] の集計もおかしい。
* [x] $a_{1}^{3}$ の計算がエラーになる。
* [ ] simple subword たちによる計算を実装する。